In [ ]:
import openpyxl
import math
quotation_temp = openpyxl.load_workbook("D:/Work/POOF/Quotation_Template.xlsx")
ws = quotation_temp.active
sheets = [ws]
product_specs = {
    "Monitor": [["7 inch screen", "Touchscreen"], 2],
    "Keyboard": [["Mechanical", "RGB Lighting", "Wireless"], 3],
    "Mouse": [["Wireless", "Ergonomic"], 2],
    "Printer": [["Color", "Laser", "Wireless", "Duplex"], 4],
    # Product with more than 16 specs
    "Laptop": [["16 inch screen", "Intel i7", "16GB RAM", "1TB SSD", "Nvidia RTX 2080", "RGB Lighting", "Wireless", "Backlit Keyboard", "Windows 10", "Bluetooth", "HDMI", "USB-C", "SD Card Reader", "Webcam", "Microphone", "Speakers", "Fingerprint Scanner", "Display Port"], 17],
    "Tablet": [["10 inch screen", "Android", "4GB RAM", "64GB Storage", "Wireless", "Bluetooth", "USB-C", "SD Card Reader", "Webcam", "Microphone", "Speakers", "Fingerprint Scanner"], 12],
    "Smartphone": [["6 inch screen", "Android", "8GB RAM", "128GB Storage", "Wireless", "Bluetooth", "USB-C", "Webcam", "Microphone", "Speakers", "Fingerprint Scanner"], 11]
}

# Sort the dictionary based on num_of_specs in descending order and overwrite the original dictionary
product_specs = dict(sorted(product_specs.items(), key=lambda item: item[1][1], reverse=True))


In [ ]:
def vacant_spots(sheet):
    vacants = 0
    num_rows_per_sheet = 8
    min_row = 14
    max_row = min_row + num_rows_per_sheet - 1
    min_col = 1
    max_col = 10
    rows = sheet.iter_rows(
        min_row=min_row, max_row=max_row, min_col=min_col, max_col=max_col
    )
    for row in rows:
        if row[0].value is None:
            vacants += 1
    return vacants

# Pseudocode for the fitting Algorithm

In [ ]:
def add_small_product_to_page(product_specs, biggest_product, ws, vacancy):

    # The product has 8 or less lines
    # Check if the product can fit in the current page
    size = product_specs[biggest_product][1] + 1
    if size > vacancy:
        raise Exception("Not enough vacancy for product sent to add_small_product_to_page")
    for product in product_specs:
        if product == biggest_product:
            # Add the product name to the first cell of the page
            ws.cell(row=14 + 8 - vacancy, column=1).value = product
            ws.cell(row=14 + 8 - vacancy, column=7).value = "Quantity"
            ws.cell(row=14 + 8 - vacancy, column=8).value = "Price"
            ws.cell(row=14 + 8 - vacancy, column=9).value = "Total"
            ws.cell(row=14 + 8 - vacancy, column=10).value = "Placeholder for product name"
            for i, spec in enumerate(product_specs[product][0]):
                ws.cell(row=15 + i + 8 - vacancy, column=1).value = spec
            break
    return True
    
    """
    # If the product has 8 or less lines, add it to the current page
    if product_specs[biggest_product][1] <= 8:
        for product in product_specs:
            if product == biggest_product:
                # Add the product name to the first cell of the page
                ws.cell(row=14, column=1).value = product
                ws.cell(row=14, column=7).value = "Quantity"
                ws.cell(row=14, column=8).value = "Price"
                ws.cell(row=14, column=9).value = "Total"
                ws.cell(row=14, column=10).value = "Placeholder for product name"
                for i, spec in enumerate(product_specs[product][0]):
                    ws.cell(row=15 + i, column=1).value = spec
                break
    """
def add_large_product_to_page(product_specs, biggest_product, ws, num_sheets):
    # If the product has more than 8 lines, split it into multiple page
    # Add the first 8 lines of the product to the first page and remove them from the dictionary
    # Repeat the process for the rest of the pages
    current_page_number = len(sheets)  # The current page number
    # Add number of sheets needed to the sheets list
    for _ in range(num_sheets):
            ws = quotation_temp.copy_worksheet(ws)
            # Empty the page
            empty_page(ws)
            # Add the new page to the sheets list
            sheets.append(ws)

    for i in range(current_page_number, current_page_number + num_sheets, 1):
        ws = sheets[i]
        for product in product_specs:
            if product == biggest_product:
                # Add the product name to the first cell of the page
                # Only add the product name to the first page
                if i == current_page_number:
                    ws.cell(row=14, column=1).value = product
                    ws.cell(row=14, column=7).value = "Quantity"
                    ws.cell(row=14, column=8).value = "Price"
                    ws.cell(row=14, column=9).value = "Total"
                    ws.cell(row=14, column=10).value = "Placeholder for product name"
                    # Add the first 7 lines of the specs to the page
                    for j in range(7):
                        ws.cell(row=15 + j, column=1).value = product_specs[product][0][j]
                    # Remove the first 7 lines from the product
                    product_specs[product][0] = product_specs[product][0][7:]
                    product_specs[product][1] -= 7
                # if it's not the first page, keep adding the specs to the page until the page is full or the product is finished
                else:
                    # If the product has less than 8 lines left, add them to the page
                    if product_specs[product][1] <= 8:
                        for j, spec in enumerate(product_specs[product][0]):
                            ws.cell(row=14 + j, column=1).value = spec
                        # Remove the specs from the product
                        product_specs[product][0] = []
                        product_specs[product][1] = 0
                        break
                    # If the product has more than 8 lines left, add the first 8 lines to the page
                    else:
                        for j in range(8):
                            ws.cell(row=14 + j, column=1).value = product_specs[product][0][j]
                        # Remove the first 8 lines from the product
                        product_specs[product][0] = product_specs[product][0][8:]
                        product_specs[product][1] -= 8
                        break
def empty_page(ws):
    # Empty the page
    for row in ws.iter_rows(min_row=14, max_row=21, min_col=1, max_col=10):
        row[0].value = None
        row[6].value = None
        row[7].value = None
        row[8].value = None
        row[9].value = None
    return ws


# Repeat the following steps until the dictionary is empty:
while len(product_specs) > 0:

    # Check current page vacancy and save the number of vacant spots in a variable vacancy
    vacancy = vacant_spots(ws)
    # Check the biggest product in the dictionary and save its size in variable size and its name in variable largest_product
    biggest_product = list(product_specs.keys())[0]
    size = product_specs[biggest_product][1] + 1
    # If size is less than 8:

    if size <= 8:
        # If vacancy is greater than or equal to size, add the product to the current page and remove it from the dictionary
        if vacancy >= size:
            add_small_product_to_page(biggest_product, ws, vacancy)
            del product_specs[biggest_product]
        # Elif vacancy is less than size:
        elif vacancy < size:
            found = False
            # Go through the rest of the products in the dictionary and check if any of them can fit in the current page
            for product in product_specs:
                temp_size = product_specs[product][1] + 1
                if temp_size <= vacancy:
                    # If a product is found, add it to the current page and remove it from the dictionary
                    add_small_product_to_page(product, ws, vacancy)
                    del product_specs[product]
                    found = True
                    break
            # If no product is found, move to the next page and add the product to the new page
            if not found:
                ws = quotation_temp.copy_worksheet(ws)
                sheets.append(ws)
                # Empty the page
                empty_page(ws)
    # Elif size is greater than 8:
    elif size > 8:
        # Create as many pages as needed to fit the product
        num_sheets = math.ceil(product_specs[biggest_product][1] / 8)
        add_large_product_to_page(biggest_product, ws, num_sheets)
        del product_specs[biggest_product]


In [ ]:
# TODO Save the workbook with the new pages in a different file
quotation_temp.save("D:/Work/POOF/Quotation_Template_with_products.xlsx")